# MARKET DATA AGENT

The purpose of this notebook is to test a tool in isolation from the entire graph to confirm proper implementation.

**NOTE**: You must have your `GOOGLE_API_KEY` defined in your .env file. You can get this API key here: https://aistudio.google.com/app/apikey

In [30]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from langchain_core.runnables import RunnableLambda
from langchain.tools import tool
from langchain_community.utilities import GoogleSerperAPIWrapper
import os
import pprint
from currensee.core import get_model, settings
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set environment variable for SERPER API key
os.environ["SERPER_API_KEY"] = "4369e38ae59aff075549b44c923813da127c06ef"


#%pip install --upgrade --quiet  langchain-community

In [31]:
from dotenv import load_dotenv

load_dotenv()

True

## Agent

An agent requires:
* the model
* tools
* a name
* a prompt describing the purpose of the tool


In [32]:
# Initialize the model
model = get_model(settings.DEFAULT_MODEL)

# Initialize Google Serper API Wrapper for data retrieval
serper_api = GoogleSerperAPIWrapper()


In [33]:
# Function to summarize the outputs from both tools
def summarize_outputs(ci_output: str, mn_output: str) -> str:
    combined_prompt = f"""
    Tool A Output (Client News and its Industry): 
    {ci_output}
    
    Tool B Output (Macroeconomic News): 
    {mn_output}
    
    Please summarize the key points from both outputs into one longer concise summary. Include specific numbers.
    """
    
    # Create the messages to pass to the model
    messages = [
        HumanMessage(content=combined_prompt)
    ]
    
    # Use the 'invoke' method for summarization
    summary = model.invoke(messages)
    
    # Access the message content correctly
    return summary.content  # Return the content of the AIMessage

# Initialize MemorySaver to keep track of the state if needed
memory_saver = MemorySaver()

# Define a React agent to invoke both tools and summarize the results
def agent_function(query_a: str, query_b: str) -> str:
    # Get the results from the tools
    output_ci = client_and_industry(query_ci)
    output_mn = macro_news(query_mn)

    # Summarize the outputs from both tools
    final_summary = summarize_outputs(output_ci, output_mn)
    return final_summary


### FINANCIAL TOOLS

In [35]:
#initial synthesized prompt?

# Parameters
client_name = "Walmart"
start_date = "1/31/2025"
end_date = "4/30/2025"
industry = "retail"
largest_holdings = ["Microsoft","Netflix","Netflix","tech sector"]
#Future: portfolio tickers(for data and news)


In [18]:
#definitions

keywords_client = ["announces", "acquires", "launches", "earnings", "report", 
               "profit", "CEO", "crisis", "disaster","recession","recovery", "red flag", 
               "urgent","challenge","emergency", "tumble","drop","opportunity","slowdown"]

keywords_econ = ["recovery","crisis", "disaster","recession","red flag", 
               "urgent","challenge","emergency", "tumble","drop","slowdown"]

trusted_sources = ["reuters.com", "bloomberg.com", "cnn.com", "forbes.com", 
                "finance.yahoo.com","marketwatch.com","morningstar.com", "https://www.wsj.com","www.ft.com"]

# Define trusted sources
allowed_sites = trusted_sources
site_filter = " OR ".join(f"site:{site}" for site in allowed_sites)

**TOOL: CLIENT AND INDUSTRY - Search for top k news articles about the client and its industry (title, snippet, date, source)**

In [19]:
# Function to retrieve news about client and its industry (Client and Industry)
@tool
def client_and_industry(query_ci: str) -> str:
    """Return the most relevant news about the client and its industry."""
    def format_google_date(date_str):
        parts = date_str.split("/")
        return f"{parts[2]}{parts[0].zfill(2)}{parts[1].zfill(2)}"

    google_start = format_google_date(start_date)
    google_end = format_google_date(end_date)

    sort_param = f"date:r:{google_start}:{google_end}"  # Google's date range format

    # Search with date filter
    search = GoogleSerperAPIWrapper(k=30, sort=sort_param)  # Pass sort parameter
    results = search.results(query_ci)

    def score_result(result):
        score = 0
        keywords = keywords_client  # Assume this variable is defined elsewhere
        link = result.get("link", "")
        title = result.get("title", "").lower()
        snippet = result.get("snippet", "").lower()

        if any(site in link for site in allowed_sites):
            score += 3

        if any(word in title or word in snippet for word in keywords):
            score += 2

        if "date" in result:
            score += 1

        return score

    if results.get("organic"):
        sorted_results_client = sorted(results.get("organic", []), key=score_result, reverse=True)
        return sorted_results_client
    else:
        return "No results found for client or industry news."



**TOOL: MACRO NEWS - Search for relevant articles about the economy (title, snippet, date, source)**

In [20]:
# Function to retrieve macroeconomic events news (Tool MACRO NEWS)
@tool
def macro_news(query_mn: str) -> str:
    """Return the most relevant macroeconomic news based on the query."""
    def format_google_date(date_str):
        parts = date_str.split("/")
        return f"{parts[2]}{parts[0].zfill(2)}{parts[1].zfill(2)}"

    google_start = format_google_date(start_date)
    google_end = format_google_date(end_date)

    sort_param = f"date:r:{google_start}:{google_end}"

    search = GoogleSerperAPIWrapper(k=30, sort=sort_param)  # Pass sort parameter
    results = search.results(query_mn)

    def score_result(result):
        score = 0
        keywords = keywords_econ  # Assume this variable is defined elsewhere
        link = result.get("link", "")
        title = result.get("title", "").lower()
        snippet = result.get("snippet", "").lower()

        if any(site in link for site in allowed_sites):
            score += 3

        if any(word in title or word in snippet for word in keywords):
            score += 2

        if "date" in result:
            score += 1

        return score

    if results.get("organic"):
        sorted_results_econ = sorted(results.get("organic", []), key=score_result, reverse=True)
        return sorted_results_econ
    else:
        return "No results found for macroeconomic events."

**TOOL: CLIENT HOLDINGS - Search for top k news articles about the client's highest exposure holdings (title, snippet, date, source)**

In [22]:
# Function to retrieve macroeconomic events news (Tool HOLDINGS NEWS)
@tool
def holdings_news(query_hn: str) -> str:
    """Return the most relevant news based on each major holding."""
    def format_google_date(date_str):
        parts = date_str.split("/")
        return f"{parts[2]}{parts[0].zfill(2)}{parts[1].zfill(2)}"

    google_start = format_google_date(start_date)
    google_end = format_google_date(end_date)

    sort_param = f"date:r:{google_start}:{google_end}"

    search = GoogleSerperAPIWrapper(k=30, sort=sort_param)  # Pass sort parameter
    results = search.results(query_hn)

    def score_result(result):
        score = 0
        keywords = keywords_client
        link = result.get("link", "")
        title = result.get("title", "").lower()
        snippet = result.get("snippet", "").lower()

        if any(site in link for site in allowed_sites):
            score += 3

        if any(word in title or word in snippet for word in keywords):
            score += 2

        if "date" in result:
            score += 1

        return score

    if results.get("organic"):
        sorted_results_econ = sorted(results.get("organic", []), key=score_result, reverse=True)
        return sorted_results_econ
    else:
        return "No results found for holdings."

**TOOL: CLIENT HOLDINGS DATA - Provide an update on the biggest movers of client portfolio**

**TOOL: MACROFIN DATA - Provide a generate market update on the latest macro/financial indicators**

### SUMMARY - CLIENT, ITS SECTOR AND MACRO NEWS

In [21]:

# Query for stock market and industry-related news
query_ci = f"{site_filter} news about {client_name} and about {industry} industry"

# Query for macroeconomic events
query_mn = f"{site_filter} news about relevant macro events and the economy."

# Run the agent and get the summary
final_summary = agent_function(query_ci, query_mn)

# Print the result
print(final_summary)

/var/tmp/ipykernel_33550/2137028457.py:30: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_ci = client_and_industry(query_ci)


Both Tool A and Tool B outputs highlight the significant impact of tariffs and economic uncertainty on the US economy and retail sector in early 2025.

**Tool A (Retail News, focusing on Walmart):**  Walmart's performance is a microcosm of the broader retail landscape. While its online business is projected to become profitable this year,  overall, Walmart's outlook is mixed.  The company's revenue reached approximately $680.47 billion for the year ending January 31, 2025 (LSEG estimates), a roughly 5% increase. However,  a lower-than-expected profit forecast has raised concerns about consumer spending.  Walmart is facing tariff challenges, pushing Chinese suppliers to cut prices to offset the impact.  The company's "Everyday Low Price" strategy is being tested, and it's actively increasing its retail media spending, mandating a 25% year-over-year increase from brands.  Despite these challenges, Walmart reaffirmed its annual sales and income forecasts.  News also indicates a shift in r

In [ ]:
### SUMMARY - HOLDINGS

In [36]:
# Query for stock market and industry-related news
query_ci = f"{site_filter} news about {client_name} and about {industry} industry"

# Query for macroeconomic events
query_mn = f"{site_filter} news about relevant macro events and the economy."

# Run the agent and get the summary
final_summary = agent_function(query_ci, query_mn)

# Print the result
print(final_summary)

TypeError: agent_function() missing 1 required positional argument: 'query_b'

Several articles discuss Netflix's partnership with Microsoft for its ad-supported streaming service, launched in July 2022.  The articles highlight Microsoft's role in providing ad technology and sales support.  While some articles speculate about a potential Microsoft acquisition of Netflix in 2023, others focus on the partnership's impact on both companies' positions in the digital advertising market.  One article notes that Netflix is shifting from heavily relying on Microsoft for its ad-supported tier to bringing more of this work in-house.  The articles span from July 2022 to May 2024, reflecting the evolving nature of the partnership and its implications.
